# PyATL
## Magic and MongoDB
### Rick Copeland, Arborian Consulting LLC 
### @rick446

# Quick question:

What does this code do?


In [1]:
def something(x):
    print 'something({})'.format(x)

y = range(5)

In [2]:
for x in y:
    something(x)

something(0)
something(1)
something(2)
something(3)
something(4)


Let me re-write that for you...

In [3]:
_iter_y = iter(y)
while True:
    try:
        x = _iter_y.next()
    except StopIteration:
        break
    something(x)
del _iter_y

something(0)
something(1)
something(2)
something(3)
something(4)


# Well, that was verbose... 

:-/

# Let's make it worse!

In [4]:
class Y(object):
    pass

y = Y()

In [5]:
for x in y:
    something(x)

TypeError: 'Y' object is not iterable

# Enter some magic

In [6]:
class Y(object):
    def __iter__(self):
        print 'Calling __iter__'
        raise TypeError("What's __iter__ to do?")
        
y = Y()

In [7]:
for x in y:
    something(x)

Calling __iter__


TypeError: What's __iter__ to do?

In [8]:
class Yiter(object):
    pass

class Y(object):
    def __iter__(self):
        return Yiter()
    
y = Y()
for x in y:
    something(x)

TypeError: iter() returned non-iterator of type 'Yiter'

In [9]:
class Yiter(object):
    def next(self):
        assert False, 'Called next()'

In [10]:
for x in y:
    something(x)

AssertionError: Called next()

# Protocols: the method behind Python magic

In [43]:
class Y(object):
    
    def __init__(self, sequence):
        self.sequence = list(sequence)
        
    def __iter__(self):
        return Yiter(self)

In [ ]:
class Yiter(object):
    
    def __init__(self, y):
        self.y = y
        self.pos = 0
        
    def next(self):
        if self.pos < len(self.y.sequence):
            self.pos += 1
            return self.y.sequence[self.pos - 1]
        else:
            raise StopIteration()

In [42]:
y = Y([1,2,3])
for x in y:
    something(x)

something(1)
something(2)
something(3)


# MongoDB Query Language

In [12]:
import pymongo
cli = pymongo.MongoClient()
db = cli.pyatl
test = db.test
test.delete_many({})

In [13]:
test.insert_many([dict(name='Document {}'.format(i), i=i) for i in range(10)])

In [14]:
test.find_one()

{u'_id': ObjectId('57ffb9dfd6697b67247da6b2'), u'i': 0, u'name': u'Document 0'}

In [15]:
test.find_one({'i': 4})

{u'_id': ObjectId('57ffb9dfd6697b67247da6b6'), u'i': 4, u'name': u'Document 4'}

In [16]:
list(test.find({'i': {'$gt': 4, '$lt': 9}}))

[{u'_id': ObjectId('57ffb9dfd6697b67247da6b7'),
  u'i': 5,
  u'name': u'Document 5'},
 {u'_id': ObjectId('57ffb9dfd6697b67247da6b8'),
  u'i': 6,
  u'name': u'Document 6'},
 {u'_id': ObjectId('57ffb9dfd6697b67247da6b9'),
  u'i': 7,
  u'name': u'Document 7'},
 {u'_id': ObjectId('57ffb9dfd6697b67247da6ba'),
  u'i': 8,
  u'name': u'Document 8'}]

In [17]:
list(test.find({'i': {'$gte': 4, '$lt': 9}}))

[{u'_id': ObjectId('57ffb9dfd6697b67247da6b6'),
  u'i': 4,
  u'name': u'Document 4'},
 {u'_id': ObjectId('57ffb9dfd6697b67247da6b7'),
  u'i': 5,
  u'name': u'Document 5'},
 {u'_id': ObjectId('57ffb9dfd6697b67247da6b8'),
  u'i': 6,
  u'name': u'Document 6'},
 {u'_id': ObjectId('57ffb9dfd6697b67247da6b9'),
  u'i': 7,
  u'name': u'Document 7'},
 {u'_id': ObjectId('57ffb9dfd6697b67247da6ba'),
  u'i': 8,
  u'name': u'Document 8'}]

Ick.

# Can we make it easier?

> Wouldn't it be nice if I could just use Python syntax to create MongoDB query documents?
> - Rick Copeland

In [18]:
import barin

In [27]:
metadata = barin.Metadata()
metadata.db = db

In [28]:
Test = barin.collection(
    metadata, 'test',
    barin.Field('_id', barin.schema.ObjectId()),
    barin.Field('name', str),
    barin.Field('i', int))


In [30]:
Test.m.find().all()

[test: {'i': 0, '_id': ObjectId('57ffb9dfd6697b67247da6b2'), 'name': u'Document 0'},
 test: {'i': 1, '_id': ObjectId('57ffb9dfd6697b67247da6b3'), 'name': u'Document 1'},
 test: {'i': 2, '_id': ObjectId('57ffb9dfd6697b67247da6b4'), 'name': u'Document 2'},
 test: {'i': 3, '_id': ObjectId('57ffb9dfd6697b67247da6b5'), 'name': u'Document 3'},
 test: {'i': 4, '_id': ObjectId('57ffb9dfd6697b67247da6b6'), 'name': u'Document 4'},
 test: {'i': 5, '_id': ObjectId('57ffb9dfd6697b67247da6b7'), 'name': u'Document 5'},
 test: {'i': 6, '_id': ObjectId('57ffb9dfd6697b67247da6b8'), 'name': u'Document 6'},
 test: {'i': 7, '_id': ObjectId('57ffb9dfd6697b67247da6b9'), 'name': u'Document 7'},
 test: {'i': 8, '_id': ObjectId('57ffb9dfd6697b67247da6ba'), 'name': u'Document 8'},
 test: {'i': 9, '_id': ObjectId('57ffb9dfd6697b67247da6bb'), 'name': u'Document 9'}]

In [32]:
Test.m.find({'i': 5}).all()

[test: {'i': 5, '_id': ObjectId('57ffb9dfd6697b67247da6b7'), 'name': u'Document 5'}]

In [34]:
Test.m.find(Test.i == 5).all()

[test: {'i': 5, '_id': ObjectId('57ffb9dfd6697b67247da6b7'), 'name': u'Document 5'}]

In [35]:
Test.m.find(Test.i > 3).all()

[test: {'i': 4, '_id': ObjectId('57ffb9dfd6697b67247da6b6'), 'name': u'Document 4'},
 test: {'i': 5, '_id': ObjectId('57ffb9dfd6697b67247da6b7'), 'name': u'Document 5'},
 test: {'i': 6, '_id': ObjectId('57ffb9dfd6697b67247da6b8'), 'name': u'Document 6'},
 test: {'i': 7, '_id': ObjectId('57ffb9dfd6697b67247da6b9'), 'name': u'Document 7'},
 test: {'i': 8, '_id': ObjectId('57ffb9dfd6697b67247da6ba'), 'name': u'Document 8'},
 test: {'i': 9, '_id': ObjectId('57ffb9dfd6697b67247da6bb'), 'name': u'Document 9'}]

In [36]:
Test.i > 3

{'i': {'$gt': 3}}

whoa.

# Let's look at the definition of `barin.field.Field`:

```
class Field(object):

    # ... lots of stuff ...

    # Query operators
    def _qop(self, op, value):
        return mql.Clause({self._name: {op: value}})

    def __eq__(self, value):
        return mql.Clause({self._name: value})

    __ne__ = partialmethod(_qop, '$ne')
    __gt__ = partialmethod(_qop, '$gt')
    __ge__ = partialmethod(_qop, '$gte')
    __lt__ = partialmethod(_qop, '$lt')
    __le__ = partialmethod(_qop, '$lte')
    in_ = partialmethod(_qop, '$in')
    nin = partialmethod(_qop, '$nin')
    exists = partialmethod(_qop, '$exists')
    type = partialmethod(_qop, '$type')
    where = partialmethod(_qop, '$where')
    geo_within = partialmethod(_qop, '$geoWithin')
    geo_intersects = partialmethod(_qop, '$geoIntersects')
    all = partialmethod(_qop, '$all')
    elem_match = partialmethod(_qop, '$elemMatch')
    size = partialmethod(_qop, '$size')
    bits_all_set = partialmethod(_qop, '$bitsAllSet')
    bits_any_set = partialmethod(_qop, '$bitsAnySet')
    bits_all_clear = partialmethod(_qop, '$bitsAllClear')
    bits_any_clear = partialmethod(_qop, '$bitsAnyClear')
    
    # ... lots of other stuff ...
```

# Aside: `partialmethod`

```
x = partialmethod(y, arg0)
```

means

```
def x(*args, **kwargs):
    return y(arg0, *args, **kwargs)
```

So the following:

```
    __lt__ = partialmethod(_qop, '$lt')
```

could also have been written as:

```
    def __lt__(self, value):
        return self._qop('$lt', value)
```

or

```
    def __lt__(self, value):
        return mql.Clause({self._name: {'$lt': value}})
```

# And now for barin.mql.Clause...

```
class Clause(dict):

    def __and__(self, other):
        return and_(self, other)

    def __or__(self, other):
        return or_(self, other)

    def __invert__(self):
        return not_(self)

```

In [38]:
from barin import mql

Test.m.find(mql.and_(Test.i.in_([1,2,3]), Test.i > 1, Test.name.regex(r'^Doc.*'))).all()

[test: {'i': 2, '_id': ObjectId('57ffb9dfd6697b67247da6b4'), 'name': u'Document 2'},
 test: {'i': 3, '_id': ObjectId('57ffb9dfd6697b67247da6b5'), 'name': u'Document 3'}]

In [41]:
Test.m.find(
    Test.i.in_([1,2,3]) &
    (Test.i > 1) & 
    Test.name.regex(r'^Doc.*')).all()

[test: {'i': 2, '_id': ObjectId('57ffb9dfd6697b67247da6b4'), 'name': u'Document 2'},
 test: {'i': 3, '_id': ObjectId('57ffb9dfd6697b67247da6b5'), 'name': u'Document 3'}]